# Setup and init

## Environment and dependencies

1. Create and activate a vitual env, e.g., `python -m venv .venv`
1. Install dependencies with `pip install -r requirements-dev.txt`
1. Copy `.env.example` and rename to `.env`.
    1. Provide the required values using the target Cosmos DB that has the conversations data you're interested in.
    1. Also provide the required Azure OpenAI values if you plan to analyze conversations data in the AI Analysis section.

# Get conversation data

In [1]:
import json
from dotenv import load_dotenv
import pandas as pd
from history import get_conversations, extend_dataframe
from inference import evaluate_chat_response
from eval_prompt import EvaluationCateogry

load_dotenv()   

True

## Date filtering

Results can be filtered by start and/or end dates. Providing a value for both will return conversations between the two dates, inclusive. Passing `None` for one and a value for the other will produce a before or after filter accordingly. Passing `None` for both will return all results. 

Note that dates can be simple, like `2024-01-30`, but can also be more targeted by providing time details, like `2024-01-30T10:15:00Z`.

In [2]:
# Get conversations for a date range (also supports only one date being provided)
start_date = pd.to_datetime("2024-01-30")
end_date = pd.to_datetime("2024-01-31")

# Uncomment to use None for start_date and end_date to get all conversations
# start_date = None
# end_date = None

## Query Cosmos DB

In [3]:
dataset = get_conversations(start_date=start_date, end_date=end_date)
df = pd.DataFrame(dataset)

row_count = df.shape[0]
print(f"{row_count} conversation turns from date range {start_date} to {end_date} found")

df.head(3)

159 conversation turns from date range 2024-01-30 00:00:00 to 2024-01-31 00:00:00 found


,id,timestamp,response_timestamp,user_query,conversation_id,context,chat_response
0,8fdd6606-2cfb-4feb-999f-9b7907488ac4,2024-01-30T23:33:21.020666,2024-01-30T23:33:40.133918,"{'id': '5e815061-db1b-6c40-e461-0edc8b8ea8ac',...",5e815061-db1b-6c40-e461-0edc8b8ea8ac,{'citations': [{'content': 'Title: Accurate Vi...,"{'id': '', 'model': 'gpt-4', 'created': 170665..."
1,90fe45a4-2c3e-4556-af91-ee0e2fa95b1c,2024-01-30T23:32:29.742239,NaN,"{'id': '7232ea15-8bc1-0299-dd24-b63772f1f678',...",7232ea15-8bc1-0299-dd24-b63772f1f678,NaN,NaN
2,9921095d-f409-4db5-a2fd-a1e52edfc59d,2024-01-30T23:11:26.711546,2024-01-30T23:11:46.184943,"{'id': '8a66c894-b425-2d81-0251-15dcb7d6bd08',...",61a0fe48-9478-e112-ebd4-c5f3c35cb5e5,{'citations': [{'content': 'Title: MSR CORE’s ...,"{'id': '', 'model': 'gpt-4', 'created': 170665..."


### Extend with calculated colums

In [4]:
# Adds some calculated columns to the dataframe like 'duration', 'turn_count' etc.
extend_dataframe(df)
df.head(3)

,id,timestamp,response_timestamp,user_query,conversation_id,context,chat_response,user_input,answer,duration,turn_count
0,8fdd6606-2cfb-4feb-999f-9b7907488ac4,2024-01-30 23:33:21.020666,2024-01-30 23:33:40.133918,"{'id': '5e815061-db1b-6c40-e461-0edc8b8ea8ac',...",5e815061-db1b-6c40-e461-0edc8b8ea8ac,{'citations': [{'content': 'Title: Accurate Vi...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",Summarize the main three points of Peter's key...,Peter Lee's keynote address at the Microsoft R...,19.113252,1
1,90fe45a4-2c3e-4556-af91-ee0e2fa95b1c,2024-01-30 23:32:29.742239,NaT,"{'id': '7232ea15-8bc1-0299-dd24-b63772f1f678',...",7232ea15-8bc1-0299-dd24-b63772f1f678,NaN,NaN,Can you summarize the key challenges tackled b...,None,NaN,1
2,9921095d-f409-4db5-a2fd-a1e52edfc59d,2024-01-30 23:11:26.711546,2024-01-30 23:11:46.184943,"{'id': '8a66c894-b425-2d81-0251-15dcb7d6bd08',...",61a0fe48-9478-e112-ebd4-c5f3c35cb5e5,{'citations': [{'content': 'Title: MSR CORE’s ...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",tell me about msr chat,Microsoft Research (MSR) has been exploring th...,19.473397,2


# Analyzing

## Save to Excel

Due the Excel's **limit of 32,767 characters per cell**, saving the DataFrame as an .xlsx file may yield unexpected and innacurate results when analyzed. Depending on the nature of your data it may be better to execute the cells in the `AI Analysis` section to analyze conversations using GPT-4.

In [ ]:
filename = 'chat_history.xlsx'

# Create a new DataFrame from the results
out_dataset = pd.DataFrame(df, columns=[
    'id', 
    'conversation_id', 
    'turn_count', 
    'timestamp', 
    'response_timestamp', 
    'duration', 
    'user_input', 
    'answer', 
    'context'
])

# Write the new DataFrame to a new Excel file
output_file_path = filename
out_dataset.to_excel(output_file_path, index=False)

out_dataset.head(4)

## AI Analysis

In [5]:
# Categories to prompt the AI to evaluate on. 
# Options: ACCURACY, RELEVANCE, COHERENCE, FLUENCY, DEPTH, INSIGHTFULNESS, OBJECTIVITY, CONTEXTUAL_APPROPRIATENESS, SENTIMENT
categories = [
    EvaluationCateogry.ACCURACY,
    EvaluationCateogry.RELEVANCE,
    EvaluationCateogry.SENTIMENT
]

num_to_process = 5
count = 0

for index, row in df.iterrows():
    try:
        evaluation, scores = evaluate_chat_response(row, categories)

        # Extend the current row with the evaluation and scores
        df.loc[index, "evaluation"] = evaluation
        df.loc[index, "scores"] = json.dumps(scores)

        # Comment these lines to process all rows
        count += 1
        if count >= num_to_process:
            break
    except Exception as e:
        print(f"\033[91mERROR processing id {row['id']}: {e}\033[0m")
        continue

df.head(5)

Evaluating chat response for 8fdd6606-2cfb-4feb-999f-9b7907488ac4 ...
Processing evaulation response ...
Evaluating chat response for 90fe45a4-2c3e-4556-af91-ee0e2fa95b1c ...
ERROR processing id 90fe45a4-2c3e-4556-af91-ee0e2fa95b1c: The user_input, context, and answer fields must be provided.
Evaluating chat response for 9921095d-f409-4db5-a2fd-a1e52edfc59d ...
Processing evaulation response ...
Evaluating chat response for f1242f7c-41f1-4608-8141-b46de131879a ...
Processing evaulation response ...
Evaluating chat response for 71375972-e0e5-470a-a301-100a4f401481 ...
Processing evaulation response ...
Evaluating chat response for eefd1c18-f176-46ca-bea6-0218f04519b6 ...
ERROR processing id eefd1c18-f176-46ca-bea6-0218f04519b6: The user_input, context, and answer fields must be provided.
Evaluating chat response for aade9ab3-38e2-401e-ba7e-66bbf7756017 ...
Processing evaulation response ...


,id,timestamp,response_timestamp,user_query,conversation_id,context,chat_response,user_input,answer,duration,turn_count,evaluation,scores
0,8fdd6606-2cfb-4feb-999f-9b7907488ac4,2024-01-30 23:33:21.020666,2024-01-30 23:33:40.133918,"{'id': '5e815061-db1b-6c40-e461-0edc8b8ea8ac',...",5e815061-db1b-6c40-e461-0edc8b8ea8ac,{'citations': [{'content': 'Title: Accurate Vi...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",Summarize the main three points of Peter's key...,Peter Lee's keynote address at the Microsoft R...,19.113252,1,The response provided does not contain any ver...,"{""accuracy"": 0, ""relevance"": 0, ""sentiment"": ""..."
1,90fe45a4-2c3e-4556-af91-ee0e2fa95b1c,2024-01-30 23:32:29.742239,NaT,"{'id': '7232ea15-8bc1-0299-dd24-b63772f1f678',...",7232ea15-8bc1-0299-dd24-b63772f1f678,NaN,NaN,Can you summarize the key challenges tackled b...,None,NaN,1,nan,nan
2,9921095d-f409-4db5-a2fd-a1e52edfc59d,2024-01-30 23:11:26.711546,2024-01-30 23:11:46.184943,"{'id': '8a66c894-b425-2d81-0251-15dcb7d6bd08',...",61a0fe48-9478-e112-ebd4-c5f3c35cb5e5,{'citations': [{'content': 'Title: MSR CORE’s ...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",tell me about msr chat,Microsoft Research (MSR) has been exploring th...,19.473397,2,The response lacks accuracy and relevance as i...,"{""accuracy"": 0, ""relevance"": 0, ""sentiment"": 3}"
3,f1242f7c-41f1-4608-8141-b46de131879a,2024-01-30 23:11:09.044242,2024-01-30 23:11:17.530884,"{'id': '61a0fe48-9478-e112-ebd4-c5f3c35cb5e5',...",61a0fe48-9478-e112-ebd4-c5f3c35cb5e5,{'citations': [{'content': 'Title: MSR CORE’s ...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",msr,It seems like your question is incomplete. Cou...,8.486642,2,The AI response is not applicable as it was pr...,"{""accuracy"": 3, ""relevance"": 3, ""sentiment"": ""..."
4,71375972-e0e5-470a-a301-100a4f401481,2024-01-30 23:06:03.088173,2024-01-30 23:06:33.381400,"{'id': '0804017d-1858-2bbd-82d7-00821fa74f1a',...",0804017d-1858-2bbd-82d7-00821fa74f1a,{'citations': [{'content': 'Transcript - Light...,"{'id': '', 'model': 'gpt-4', 'created': 170665...",Can you summarize the key challenges tackled b...,Microsoft Research (MSR) addressed several key...,30.293227,1,The response contains fabricated references (e...,"{""accuracy"": 0, ""relevance"": 0, ""sentiment"": ""..."
